In [61]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge, RidgeCV, LassoCV
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score
import warnings

warnings.filterwarnings('ignore')

In [2]:
RANDOM_STATE = 42

In [3]:
dataset = load_boston()
X = pd.DataFrame(dataset.data)
X.columns = dataset.feature_names
y = dataset.target

1. Разделите выборку на обучающую и тестовую в отношении 80%/20%

In [4]:
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

2. Обучите стандартную регрессию, а также Ridge и  Lasso и параметрами по умолчанию и выведите их R2 на тестовой выборке

In [5]:
model_linear = LinearRegression().fit(X_train, y_train)
model_linear.score(X_test, y_test)

0.6687594935356318

In [6]:
model_ridge = Ridge().fit(X_train, y_train)
model_ridge.score(X_test, y_test)

0.6662221670168519

In [7]:
model_lasso = Lasso().fit(X_train, y_train)
model_lasso.score(X_test, y_test)

0.6671453631686304

3. Для Ridge и Lasso подберите коэффициент регуляризации(используйте GridSearchCV, RidgeCV, LassoCV) в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по лучшим моделям и сравните с предыдущими результатами. Напишите как изменился результат

In [8]:
parameters = [
    {'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1_000, 10_000, 10_000]}
]

### GridSearchCV

In [9]:
estimator_ridge = GridSearchCV(Ridge(), parameters)
estimator_ridge.fit(X_train, y_train)
estimator_ridge.best_params_

{'alpha': 1e-05}

In [10]:
estimator_ridge.score(X_test, y_test)
# R2 увеличился

0.6687594856409733

In [11]:
estimator_lasso = GridSearchCV(Lasso(), parameters)
estimator_lasso.fit(X_train, y_train)
estimator_lasso.best_params_

{'alpha': 1e-05}

In [12]:
estimator_lasso.score(X_test, y_test)
# R2 увеличился (лучше)

0.6687598638315155

### RidgeCV

In [13]:
model_ridgecv = RidgeCV(alphas=(1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1_000, 10_000, 10_000))
model_ridgecv.fit(X_train, y_train)
model_ridgecv.score(X_test, y_test)
# R2 увеличился (лучше)

0.6687510090373743

### LassoCV

In [14]:
model_lassocv = LassoCV(alphas=(1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1_000, 10_000, 10_000))
model_lassocv.fit(X_train, y_train)
model_lassocv.score(X_test, y_test)
# R2 увеличился (лучше)

0.6687598638315155

4. Проведите масштабирование выборки(используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [51]:
pipe_ridge_standard_scaler = Pipeline(
    [('scaler', StandardScaler()), ('model', Ridge())]
)
pipe_ridge_standard_scaler.fit(X_train, y_train)
pipe_ridge_standard_scaler.score(X_test, y_test)
# R2, по сравнению с Ridge(), увеличился (лучше)

0.6684624359643561

In [52]:
pipe_lasso_standard_scaler = Pipeline(
    [('scaler', StandardScaler()), ('model', Lasso())]
)
pipe_lasso_standard_scaler.fit(X_train, y_train)
pipe_lasso_standard_scaler.score(X_test, y_test)
# R2, по сравнению с Lasso(), заметно уменьшился

0.6239428734251422

In [54]:
pipe_ridge_minmax_scaler = Pipeline(
    [('scaler', MinMaxScaler()), ('model', Ridge())]
)
pipe_ridge_minmax_scaler.fit(X_train, y_train)
pipe_ridge_minmax_scaler.score(X_test, y_test)
# R2 больше чем у Ridge(), но меньше чем с StandardScaler()

0.6764100365423598

In [56]:
pipe_lasso_minmax_scaler = Pipeline(
    [('scaler', MinMaxScaler()), ('model', Lasso())]
)
pipe_lasso_minmax_scaler.fit(X_train, y_train)
pipe_lasso_minmax_scaler.score(X_test, y_test)
# R2 значительное уменьшение по сравнению как с чистым Lasso(), так и с StandardScaler() 

0.2573921442545195

5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [30]:
pipe_ridgecv_standard_scaler = Pipeline(
    [('scaler', StandardScaler()), ('model', RidgeCV(alphas=(1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1_000, 10_000, 10_000)))]
)
pipe_ridgecv_standard_scaler.fit(X_train, y_train)
pipe_ridgecv_standard_scaler.score(X_test, y_test)
# R2, по сравнению с чистым RidgeCV(), уменьшился

0.6659677905050798

In [57]:
pipe_lassocv_standard_scaler = Pipeline(
    [('scaler', StandardScaler()), ('model', LassoCV(alphas=(1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1_000, 10_000, 10_000)))]
)
pipe_lassocv_standard_scaler.fit(X_train, y_train)
pipe_lassocv_standard_scaler.score(X_test, y_test)
# R2, по сравнению с чистым LassoCV(), немного уменьшился

0.668759038334717

In [58]:
pipe_ridgecv_minmax_scaler = Pipeline(
    [('scaler', MinMaxScaler()), ('model', RidgeCV(alphas=(1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1_000, 10_000, 10_000)))]
)
pipe_ridgecv_minmax_scaler.fit(X_train, y_train)
pipe_ridgecv_minmax_scaler.score(X_test, y_test)
# R2, по сравнению как с чистым RidgeCV(), так и с StandardScaler() заметно увеличился

0.6700309977617445

In [60]:
pipe_lassocv_minmax_scaler = Pipeline(
    [('scaler', MinMaxScaler()), ('model', LassoCV(alphas=(1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1_000, 10_000, 10_000)))]
)
pipe_lassocv_minmax_scaler.fit(X_train, y_train)
pipe_lassocv_minmax_scaler.score(X_test, y_test)
# R2, по сравнению как с чистым LassoCV(), так и с StandardScaler() немного увеличился

0.6687605073677362

6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [66]:
pipe_ridge_standard_scaler_polynomial = Pipeline(
    [('scaler', StandardScaler()), ('polynomial', PolynomialFeatures(2)), ('model', Ridge())]
)
pipe_ridge_standard_scaler_polynomial.fit(X_train, y_train)
pipe_ridge_standard_scaler_polynomial.score(X_test, y_test)
# R2 значительное увеличение по сравнению с предыдущими Ridge

0.816294899023

In [67]:
pipe_lasso_standard_scaler_polynomial = Pipeline(
    [('scaler', StandardScaler()), ('polynomial', PolynomialFeatures(2)), ('model', Lasso())]
)
pipe_lasso_standard_scaler_polynomial.fit(X_train, y_train)
pipe_lasso_standard_scaler_polynomial.score(X_test, y_test)
# R2 значительное увеличение по сравнению с предыдущими Lasso

0.7322762586137164

In [70]:
pipe_ridge_minmax_scaler_polynomial = Pipeline(
    [('scaler', MinMaxScaler()), ('polynomial', PolynomialFeatures(2)), ('model', Ridge())]
)
pipe_ridge_minmax_scaler_polynomial.fit(X_train, y_train)
pipe_ridge_minmax_scaler_polynomial.score(X_test, y_test)
# R2 увеличение по сравнению с предыдущими Ridge

0.8299337208033138

In [71]:
pipe_lasso_minmax_scaler_polynomial = Pipeline(
    [('scaler', MinMaxScaler()), ('polynomial', PolynomialFeatures(2)), ('model', Lasso())]
)
pipe_lasso_minmax_scaler_polynomial.fit(X_train, y_train)
pipe_lasso_minmax_scaler_polynomial.score(X_test, y_test)
# R2 значительное уменьшение по сравнению как с чистым Lasso(), так и с StandardScaler() 

0.2611262741735658

In [72]:
pipe_ridgecv_standard_scaler_polynomial = Pipeline(
    [('scaler', StandardScaler()),
     ('polynomial', PolynomialFeatures(2)),
     ('model', RidgeCV(alphas=(1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1_000, 10_000, 10_000)))]
)
pipe_ridgecv_standard_scaler_polynomial.fit(X_train, y_train)
pipe_ridgecv_standard_scaler_polynomial.score(X_test, y_test)
# R2 значительное увеличение по сравнению с предыдущими RidgeCV()

0.8180465877243803

In [74]:
pipe_lassocv_standard_scaler_polynomial = Pipeline(
    [('scaler', StandardScaler()),
     ('polynomial', PolynomialFeatures(2)),
     ('model', LassoCV(alphas=(1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1_000, 10_000, 10_000)))]
)
pipe_lassocv_standard_scaler_polynomial.fit(X_train, y_train)
pipe_lassocv_standard_scaler_polynomial.score(X_test, y_test)
# R2 значительное увеличение по сравнению с предыдущими LassoCV()

0.8122168137984359

In [84]:
pipe_ridgecv_minmax_scaler_polynomial = Pipeline(
    [('scaler', MinMaxScaler()),
     ('polynomial', PolynomialFeatures(2)),
     ('model', RidgeCV(alphas=(1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1_000, 10_000, 10_000)))]
)
pipe_ridgecv_minmax_scaler_polynomial.fit(X_train, y_train)
pipe_ridgecv_minmax_scaler_polynomial.score(X_test, y_test)
# R2 значительное увеличение по сравнению с предыдущими RidgeCV()

0.8500630422288056

In [77]:
pipe_lassocv_minmax_scaler_polynomial = Pipeline(
    [('scaler', MinMaxScaler()),
     ('polynomial', PolynomialFeatures(2)),
     ('model', LassoCV(alphas=(1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1_000, 10_000, 10_000)))]
)
pipe_lassocv_minmax_scaler_polynomial.fit(X_train, y_train)
pipe_lassocv_minmax_scaler_polynomial.score(X_test, y_test)
# R2 значительное увеличение по сравнению с предыдущими LassoCV()

0.8390581680518306

In [79]:
pipe_linear = Pipeline ( [('scaler', MinMaxScaler()), ('polynomial', PolynomialFeatures(2)), ('model', LinearRegression())])
pipe_linear.fit(X_train, y_train)
pipe_linear.score(X_test, y_test)

0.8033778680650641

7. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2. Напишите как изменился R2 по сравнению с предыдущими экспериментами

http://archive.ics.uci.edu/ml/datasets/Adult

In [ ]:
link = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/adult-all.csv'
data = pd.read_csv(link, header=None)

In [ ]:
data.head()

8. Разделите выборку на признаки и целевую переменную(колонка со зачениями {<=50K,>50K}). Замените целевую переменную на числовые значения.

9. Выясните, присутствуют ли в данных пропуски. Заполните их самыми частыми значениями (испольуйте SimpleImputer)

10. Выберите колонки с числовыми и категориальными переменными.

11. Создайте пайплайн по обработке колонок(используйте OneHotEncoder,MinMaxScaler).

12. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

13. Посчитайте cross_val_score по алгоритмам LogisticRegression, SVC, LinearSVC по метрикам accuracy и f1_score.
Напишите удалось ли превзойти предыдущий результат.

14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями (испольуйте SimpleImputer)

15. Посчитайте cross_val_score на новых данных. Напишите удалось ли улучшить результат.

16. Посчитайте cross_val_score, если просто удалить значения '?'. Напишите как изменился результат

 17. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier. Напишите как изменился результат и какой вывод можно из этого сделать.

18. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.